In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    # torch_dtype=torch.float32,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    # attn_implementation="sdpa",
    attn_implementation="flash_attention_2",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

# correct_dataset = correct_dataset[:30]
len(correct_dataset)

'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
209 185 0.8851674641148325


185

In [2]:
import torch
import gc
from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm
from tqdm import tqdm as text_tqdm
from hidden_capacity_reasoning.utils import tokenize_single_turn

torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

for dataset_pos in tqdm(range(1, len(correct_dataset))):
    tokenized_turn = tokenize_single_turn(
        question=base_prompt.format(question=correct_dataset[dataset_pos]["problem"]),
        answer=correct_dataset[dataset_pos]["model_answer"],
        tokenizer=tokenizer,
    )
    break

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
tokenized_turn

In [33]:
for key in tokenized_turn.keys():
    tokenized_turn[key] = torch.tensor(tokenized_turn[key])

device = "cuda"

content_compression_mask = tokenized_turn["content_compression_mask"]

input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
# get only question part
question_input_ids = (
    tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0).cuda()
)
tokenizer.decode(question_input_ids[-1])

/tmp/ipykernel_21672/4083140599.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_turn[key] = torch.tensor(tokenized_turn[key])


'<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \\ldots, 99, 100 ?$\n\nPlease reason step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\n'

In [ ]:
tokenizer.decode(tokenized_turn["input_ids"][content_compression_mask == 2])

'Okay, so I have this problem here: "How many numbers are in the list 25, 26, 27, ..., 99, 100?" Hmm, let me think about how to approach this. I remember that when you have a sequence of numbers starting from a certain number and going up by 1 each time, you can find the total number of terms using a formula. I think it\'s something like the last term minus the first term plus one. Let me verify that.\n\nSo, if I have a sequence like 1, 2, 3, ..., n, the number of terms is n. But in this case, the sequence starts at 25 and ends at 100. So, if I subtract 25 from 100, that gives me 75. But wait, that doesn\'t seem right because 25 to 100 is a lot more numbers than 75. Maybe I need to add something else.\n\nLet me recall the formula for the number of terms in an arithmetic sequence. I think it\'s (last term - first term) divided by the common difference plus one. Since the common difference here is 1 (because each number is one more than the previous), the formula simplifies to just (last

In [35]:
tokenizer.decode(tokenized_turn["input_ids"][content_compression_mask == 3])

'\n\nTo determine how many numbers are in the list \\(25, 26, 27, \\ldots, 99, 100\\), we can use the formula for the number of terms in an arithmetic sequence. The formula is:\n\n\\[\n\\text{Number of terms} = \\left( \\text{Last term} - \\text{First term} \\right) / \\text{Common difference} + 1\n\\]\n\nHere, the first term is 25, the last term is 100, and the common difference is 1. Plugging these values into the formula, we get:\n\n\\[\n\\text{Number of terms} = \\left( 100 - 25 \\right) / 1 + 1 = 75 + 1 = 76\n\\]\n\nThus, the number of numbers in the list is \\(\\boxed{76}\\).'

In [11]:
tokenized_turn["input_ids"][content_compression_mask == 2].shape

torch.Size([1568])

In [ ]:
tokenized_turn["input_ids"][content_compression_mask == 2]

{'input_ids': tensor([151646, 151644,  31198,  ...,  11035,    568, 151643]),
 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'content_compression_mask': tensor([0, 0, 1,  ..., 3, 3, 0])}

In [ ]:
# block_size = 4
# think_block_size = tokenized_turn["input_ids"][content_compression_mask == 2].shape[0]
# block_mask = torch.arange(0, block_size).repeat(
#     think_block_size // block_size + 1,
# )
# block_mask = block_mask[:think_block_size]
#
# tokenized_turn["input_ids"][input_part_end : input_part_end + think_block_size][
#     block_mask != 0
# ] = 151662

In [17]:
tokenizer.decode([151662])

'<|fim_pad|>'

In [16]:
tokenizer.decode(tokenized_turn["input_ids"])

'<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \\ldots, 99, 100 ?$\n\nPlease reason step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\nOkay, so I have this problem here: "How many numbers are in the list 25, 26, 27, ..., 99, 100?" Hmm, let me think about how to approach this. I remember that when you have a sequence of numbers starting from a certain number and going up by 1 each time, you can find the total number of terms using a formula. I think it\'s something like the last term minus the first term plus one. Let me verify that.\n\nSo, if I have a sequence like 1, 2, 3, ..., n, the number of terms is n. But in this case, the sequence starts at 25 and ends at 100. So, if I subtract 25 from 100, that gives me 75. But wait, that doesn\'t seem right because 25 to 100 is a lot more numbers than 75. Maybe I need to add something else.\n\nLet me recall the formula for the number of terms in an arithmetic sequence. I think i

In [39]:
block_size = 16
# think_block_size = tokenized_turn["input_ids"][content_compression_mask == 2].shape[0]
think_block_size = tokenized_turn["input_ids"].shape[0] - input_part_end - 1
block_mask = torch.ones(think_block_size)


original_input_ids = tokenized_turn["input_ids"].clone()
for block_pos in range(1, block_mask.shape[0] // block_size + 2):
    train_block_size = block_pos * block_size
    # temp_mask_1 = torch.ones(think_block_size)
    temp_block_mask = block_mask.clone()
    temp_block_mask[train_block_size:] = block_size

    if block_pos > 1:
        temp_block_mask[: (block_pos - 1) * block_size] = block_size

    temp_input_ids = original_input_ids.clone()
    temp_input_ids[input_part_end + 1 : input_part_end + 1 + think_block_size][
        temp_block_mask != block_size
    ] = 151662
    # print(block_mask)
    print(tokenizer.decode(temp_input_ids))
    print("==")
    print("==")
    print("==")
    # break

<｜begin▁of▁sentence｜><｜User｜>Problem: How many numbers are in the list $25, 26, 27, \ldots, 99, 100 ?$

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>
<|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|> list 25, 26, 27, ..., 99, 100?" Hmm, let me think about how to approach this. I remember that when you have a sequence of numbers starting from a certain number and going up by 1 each time, you can find the total number of terms using a formula. I think it's something like the last term minus the first term plus one. Let me verify that.

So, if I have a sequence like 1, 2, 3, ..., n, the number of terms is n. But in this case, the sequence starts at 25 and ends at 100. So, if I subtract 25 from 100, that gives me 75. But wait, that doesn't seem right because 25 to 100 is a lot more numbers than 75. Maybe I need to add so

In [ ]:
def multiple_token_prediction_data_v1(tokenized_turn, block_size=16, tokenizer=None):
    for key in tokenized_turn.keys():
        tokenized_turn[key] = torch.tensor(tokenized_turn[key])

    content_compression_mask = tokenized_turn["content_compression_mask"]

    input_part_end = (content_compression_mask == 0).nonzero()[-3][0]
    # get only question part
    question_input_ids = (
        tokenized_turn["input_ids"][: int(input_part_end) + 1].unsqueeze(0).cuda()
    )

    # think_block_size = tokenized_turn["input_ids"][content_compression_mask == 2].shape[0]
    think_block_size = tokenized_turn["input_ids"].shape[0] - input_part_end - 1
    block_mask = torch.ones(think_block_size)

    original_input_ids = tokenized_turn["input_ids"].clone()
    tokens_dataset = []
    for block_pos in range(1, block_mask.shape[0] // block_size + 2):
        train_block_size = block_pos * block_size
        # temp_mask_1 = torch.ones(think_block_size)
        temp_block_mask = block_mask.clone()
        temp_block_mask[train_block_size:] = block_size

        if block_pos > 1:
            temp_block_mask[: (block_pos - 1) * block_size] = block_size

        temp_input_ids = original_input_ids.clone()
        temp_input_ids[input_part_end + 1 : input_part_end + 1 + think_block_size][
            temp_block_mask != block_size
        ] = 151662

        labels = tokenized_turn["input_ids"].clone()
        labels[content_compression_mask == 1] = 151662
        tokens_dataset.append(
            {
                "input_ids": temp_input_ids.tolist(),
                "labels": labels.tolist(),
            }
        )
    return tokens_dataset


data = multiple_token_prediction_data_v1(tokenized_turn)
##

In [60]:
data[0]

{'input_ids': tensor([[[151646, 151644,  31198,  ...,  11035,    568, 151643]]],
        device='cuda:0'),
 'labels': tensor([[[151646, 151644,   -100,  ...,  11035,    568, 151643]]],
        device='cuda:0')}

In [66]:
import copy

with torch.no_grad():
    item = copy.copy(data[0])
    for key in item.keys():
        item[key] = torch.tensor(
            item[key],
            device=model.device,
        ).unsqueeze(0)
    loss = model(**item).loss
    print(loss)

tensor(0.5182, device='cuda:0')


In [ ]:
item["input_ids"].shape

torch.Size([1822])

In [ ]:
item["labels"].shape

torch.Size([1822])

#### create batch

In [5]:
import copy

batch_size = 2
batch = copy.copy(data[:batch_size])
inputs_str = [tokenizer.decode(item["input_ids"]) for item in batch]
labels = [tokenizer.decode(item["labels"]) for item in batch]
# batch
inputs_str = tokenizer(
    inputs_str,
    padding="longest",
    return_tensors="pt",
)
labels = tokenizer(
    labels,
    padding="longest",
    return_tensors="pt",
)
inputs_str["labels"] = labels["input_ids"]

In [74]:
inputs_str

{'input_ids': tensor([[151646, 151646, 151644,  ...,  11035,    568, 151643],
        [151646, 151646, 151644,  ...,  11035,    568, 151643],
        [151646, 151646, 151644,  ...,  11035,    568, 151643],
        ...,
        [151646, 151646, 151644,  ...,  11035,    568, 151643],
        [151646, 151646, 151644,  ...,  11035,    568, 151643],
        [151646, 151646, 151644,  ...,  11035,    568, 151643]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [6]:
import copy

with torch.no_grad():
    inputs_str = inputs_str.to(model.device)
    loss = model(**inputs_str).loss
    print(loss)

tensor(1.1042, device='cuda:0')


# Debug error

In [4]:
import os

os.environ["WANDB_PROJECT"] = "multiple_tokens_prediction_math_500"
os.environ["TOKENIZERS_PARALLELISM"] = "true"
from transformers import Qwen2ForCausalLM, Qwen2Model, AutoTokenizer, BitsAndBytesConfig
import torch
from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
)

from datasets import load_dataset
from tqdm import tqdm
from hidden_capacity_reasoning.utils import (
    generate_train_examples,
    pad_train_examples,
    tokenize_single_turn,
)
from datasets import Dataset, IterableDataset
import gc
import types

# need for auto SFTTrainer patch(possible increase speed)
from unsloth import is_bfloat16_supported
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from hidden_capacity_reasoning.utils import (
    EOS_TOKEN_ID,
    TEXT_TOKEN_ID,
)

import time
from datetime import datetime


from transformers import AutoModelForCausalLM

from torch.utils.data import Dataset
from joblib import Parallel, delayed
from tqdm.contrib.concurrent import process_map
from tqdm_joblib import tqdm_joblib
from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
import datasets


class CustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        return self.dataset[idx]


BLOCK_SIZE = 16


def multiple_token_prediction_data_v1(
    tokenized_turn,
    block_size=None,
):
    """
    последовательность не заканчивается после маскированных токенов
    """
    for key in tokenized_turn.keys():
        tokenized_turn[key] = torch.tensor(tokenized_turn[key])

    content_compression_mask = tokenized_turn["content_compression_mask"]

    input_part_end = (content_compression_mask == 0).nonzero()[-3][0]

    think_block_size = tokenized_turn["input_ids"].shape[0] - input_part_end - 1
    block_mask = torch.ones(think_block_size)

    original_input_ids = tokenized_turn["input_ids"].clone()
    tokens_dataset = []
    for block_pos in range(1, block_mask.shape[0] // block_size + 2):
        train_block_size = block_pos * block_size
        # temp_mask_1 = torch.ones(think_block_size)
        temp_block_mask = block_mask.clone()
        temp_block_mask[train_block_size:] = block_size

        if block_pos > 1:
            temp_block_mask[: (block_pos - 1) * block_size] = block_size

        temp_input_ids = original_input_ids.clone()
        temp_input_ids[input_part_end + 1 : input_part_end + 1 + think_block_size][
            temp_block_mask != block_size
        ] = TEXT_TOKEN_ID

        labels = tokenized_turn["input_ids"].clone()
        labels[content_compression_mask == 1] = TEXT_TOKEN_ID
        tokens_dataset.append(
            {
                "input_ids": temp_input_ids.tolist(),
                "labels": labels.tolist(),
            }
        )
    return tokens_dataset


def multiple_token_prediction_data_v2(
    tokenized_turn,
    block_size=None,
):
    """
    последовательность заканчивается после маскированных токенов
    """
    for key in tokenized_turn.keys():
        tokenized_turn[key] = torch.tensor(tokenized_turn[key])

    content_compression_mask = tokenized_turn["content_compression_mask"]

    input_part_end = (content_compression_mask == 0).nonzero()[-3][0]

    think_block_size = tokenized_turn["input_ids"].shape[0] - input_part_end - 1
    block_mask = torch.ones(think_block_size)

    original_input_ids = tokenized_turn["input_ids"].clone()
    tokens_dataset = []
    for block_pos in range(1, block_mask.shape[0] // block_size + 2):
        train_block_size = block_pos * block_size
        # temp_mask_1 = torch.ones(think_block_size)
        temp_block_mask = block_mask.clone()
        temp_block_mask[train_block_size:] = block_size

        if block_pos > 1:
            temp_block_mask[: (block_pos - 1) * block_size] = block_size

        temp_input_ids = original_input_ids.clone()
        temp_input_ids[input_part_end + 1 : input_part_end + 1 + think_block_size][
            temp_block_mask != block_size
        ] = TEXT_TOKEN_ID
        temp_input_ids = temp_input_ids[: input_part_end + 1 + train_block_size]

        labels = tokenized_turn["input_ids"].clone()
        labels[content_compression_mask == 1] = TEXT_TOKEN_ID
        labels = labels[: input_part_end + 1 + train_block_size]
        tokens_dataset.append(
            {
                "input_ids": temp_input_ids.tolist(),
                "labels": labels.tolist(),
            }
        )
    return tokens_dataset


# def main():
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map={"": 0},
#     attn_implementation="flash_attention_2",
# )
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset(
    "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

base_prompt = open(
    "/code/hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

dataset = dataset["train"].train_test_split(
    test_size=5,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

dataset = dataset.rename_columns(
    {
        "problem": "question",
        "answer": "gold_answer",
        "model_answer": "answer",
    }
)
dataset = dataset.remove_columns(
    [item for item in dataset.column_names if not item in ["question", "answer"]]
)

# test pass
tokenize_single_turn(
    question=base_prompt.format(question=dataset[0]["question"]),
    answer=dataset[0]["answer"],
    tokenizer=tokenizer,
)
train_examples = [
    tokenize_single_turn(
        tokenizer=tokenizer,
        question=base_prompt.format(question=item["question"]),
        answer=item["answer"],
    )
    for item in tqdm(dataset.to_list())
    # for item in tqdm(dataset.to_list()[:2000])
    # for item in tqdm(dataset.to_list()[:1])
]

# prepared_train_examples = []
# with tqdm_joblib(
#     tqdm(desc="My calculation", total=len(train_examples))
# ) as progress_bar:
#     examples = Parallel(n_jobs=-1)(
#         delayed(multiple_token_prediction_data_v2)(
#             tokenized_turn=tokenized_turn,
#             block_size=BLOCK_SIZE,
#         )
#         for tokenized_turn in train_examples
#     )
# for example in examples:
#     for item in example:
#         prepared_train_examples.append(item)

# # new_dataset = Dataset.from_list(prepared_train_examples)
# new_dataset = CustomDataset(prepared_train_examples)
# print(dataset)
temp_data = multiple_token_prediction_data_v2(
    tokenized_turn=train_examples[0],
    block_size=BLOCK_SIZE,
)

100%|██████████| 4/4 [00:00<00:00, 328.37it/s]


In [10]:
# tokenizer.decode(temp_data[40]["labels"])
tokenizer.decode(temp_data[40]["input_ids"])

"<｜begin▁of▁sentence｜><｜User｜>Problem: If $A=2+i$, $O=-4$, $P=-i$, and $S=2+4i$, find $A-O+P+S$.\n\nPlease reason step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\nOkay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I 

In [29]:
from tqdm.notebook import tqdm

for batch in tqdm(new_dataset):
    # if len(item['input_ids']) != len(item['labels']):
    #     print(item)
    #     break
    # только для batch=1
    batch = [batch]
    inputs_str = [{"input_ids": item["input_ids"]} for item in batch]
    labels = [{"input_ids": item["labels"]} for item in batch]
    # batch
    padded_batch = tokenizer.pad(
        inputs_str,
        padding="longest",
        return_tensors="pt",
    )
    labels_new = tokenizer.pad(
        labels,
        padding="longest",
        return_tensors="pt",
    )
    padded_batch["labels"] = labels_new["input_ids"]

    skip_ids = [
        TEXT_TOKEN_ID,
        EOS_TOKEN_ID,
    ]
    for skip_id in skip_ids:
        padded_batch["labels"][padded_batch["labels"] == skip_id] = -100
    if padded_batch["labels"].shape[1] != padded_batch["input_ids"].shape[1]:
        print(padded_batch)
        break
    # break
padded_batch["labels"].shape[1], padded_batch["input_ids"].shape[1]

  0%|          | 0/28226 [00:00<?, ?it/s]

(4189, 4189)

In [20]:
labels

["<｜begin▁of▁sentence｜><｜User｜><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><|fim_pad|><｜Assistant｜><think>\nOkay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I n

In [ ]:
labels_new["input_ids"].shape

torch.Size([1, 2415])

In [ ]:
padded_batch["input_ids"].shape

torch.Size([1, 2414])

In [ ]:
len(item["input_ids"]) == len(item["labels"])

True

In [ ]:
temp_labels = [{"input_ids": item["labels"]} for item in batch]
tokenizer.pad(
    temp_labels,
    pad_to_multiple_of=8,
    padding="longest",
)["input_ids"]

[[151643,
  151643,
  151646,
  151644,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151662,
  151645,
  151648,
  198,
  32313,
  11,
  773,
  358,
  614,
  419,
  3491,
  1588,
  1380,
  358,
  1184,
  311,
  1477,
  279,
  897,
  315,
  362,
  27283,
  506,
  5519,
  393,
  5519,
  328,
  13,
  576,
  2661,
  2750,
  525,
  362,
  284,
  220,
  17,
  488,
  600,
  11,
  506,
  284,
  481,
  19,
  11,
  393,
  284,
  481,
  72,
  11,
  323,
  328,
  284,
  220,
  17,
  488,
  220,
  19,
  72,
  13,
  88190,
  11,
  1077,
  75

### inference

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "multiple_tokens/2025_06_06_21_30_00_076194/checkpoint-240000"
# model_name = "multiple_tokens/2025_06_06_01_58_40_209898/checkpoint-60000"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,
    device_map={"": 0},
    attn_implementation="sdpa",
)
# model = model.eval()
model.requires_grad_(False)

from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    # "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    # test_size=250,
    test_size=350,
    # test_size=999,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)


correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

print(len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset))

correct_dataset = correct_dataset[:30]
len(correct_dataset)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
224 202 0.9017857142857143


30

In [2]:
correct_dataset[0]

{'problem': 'Below is a magic square, meaning that the sum of the numbers in each row, in each column, and in each of the $2$ main diagonals are equal. What is the value of $n$?\n\n[asy]size(125);\nfor(int i = 0; i<4; ++i)\n{\n\ndraw((0,i)--(3,i),linewidth(1));\n}\n\nfor(int j = 0; j<4; ++j)\n{\n\ndraw((j,0)--(j,3),linewidth(1));\n}\n\nlabel("$n-3$",(.5,.5));\nlabel("3",(.5,1.5));\nlabel("$n+1$",(.5,2.5));\n\nlabel("$n+2$",(1.5,.5));\nlabel("$2n-9$",(1.5,1.5));\nlabel("$1$",(1.5,2.5));\n\nlabel("$2$",(2.5,.5));\nlabel("$n$",(2.5,1.5));\nlabel("$n-1$",(2.5,2.5));\n[/asy]',
 'solution': 'First, we can evaluate the sum across the first row, which gives $(n+1)+1+(n-1)=2n+1$.  Evaluate the sum of the entries across the second row, $3+(2n-9)+n=3n-6$. Now, since we have a magic square, these two sums are equal.  So $2n+1=3n-6$. Isolating $n$, we obtain $n = \\boxed{7}$.\n\nThe square will look like: [asy] size(2cm);\ndraw((0,0)--(3,0)--(3,3)--(0,3)--cycle,linewidth(1));\ndraw((1,0)--(1,3),lin

In [22]:
from tqdm.notebook import tqdm
from hidden_capacity_reasoning.utils import (
    generate_train_examples,
    pad_train_examples,
    tokenize_single_turn,
)
from hidden_capacity_reasoning.utils import (
    EOS_TOKEN_ID,
    TEXT_TOKEN_ID,
)

base_prompt = open(
    "/code/hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()
# test_examples = [
#     tokenize_single_turn(
#         tokenizer=tokenizer,
#         question=base_prompt.format(question=item["problem"]),
#         answer=item["model_answer"],
#     )
#     for item in tqdm(correct_dataset)
#     # for item in tqdm(dataset.to_list()[:2000])
#     # for item in tqdm(dataset.to_list()[:1])
# ]
max_correct = 7
original_tokens = tokenizer.encode(
    item["model_answer"],
    add_special_tokens=False,
)
for item in correct_dataset:
    model_input = base_prompt.format(question=item["problem"])
    input_ids = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": model_input},
        ],
        add_generation_prompt=True,
        tokenize=False,
    )
    # print(input_ids)
    input_ids = tokenizer(
        input_ids,
        return_tensors="pt",
        add_special_tokens=False,
    ).to("cuda")
    input_ids = input_ids["input_ids"]
    # print(input_ids.shape)
    PAD_SIZE = 16
    pad_part = torch.ones(1, PAD_SIZE) * TEXT_TOKEN_ID
    pad_part = pad_part.long().cuda()
    # input_ids = torch.cat(
    #     [
    #         input_ids,
    #         pad_part,
    #     ],
    #     dim=1,
    # )
    # # print(input_ids.shape)
    # print("ORIGINAL", tokenizer.decode(input_ids[-1]))
    # total_parts = len(original_tokens) // PAD_SIZE - 1
    total_parts = 5
    for part_num in range(total_parts):
        temp_input_ids = torch.cat(
            [
                input_ids,
                pad_part,
            ],
            dim=1,
        )
        # print('temp_input_ids',tokenizer.decode(temp_input_ids[-1]))
        # print(tokenizer.decode(input_ids[-1]))
        with torch.no_grad():
            model_predicts = model(
                input_ids=temp_input_ids,
            )
        labels = model_predicts.logits.argmax(-1)
        if part_num == 0:
            labels = labels[:, -PAD_SIZE - 1 :][:, :max_correct]
        else:
            labels = labels[:, -PAD_SIZE:][:, :max_correct]

        labels = labels[:, -PAD_SIZE:][:, :max_correct]
        print(tokenizer.decode(labels[-1]))
        # print(tokenizer.decode(original_tokens))
        print("===")
        print("===")
        print("===")
        input_ids = torch.cat(
            [
                input_ids,
                labels,
            ],
            dim=1,
        )
        # break
    break

Okay, so I have this magic
===
===
===
 problem and I I each each each
===
===
===
 the the the. the the the
===
===
===
 of the the the the the the
===
===
===
 of the... the the
===
===
===


In [4]:
item["model_answer"]

"Okay, so I have this magic square problem here, and I need to find the value of \\( n \\). Let me try to figure this out step by step. \n\nFirst, I remember that a magic square is a grid where the sums of numbers in each row, each column, and both main diagonals are equal. That common sum is called the magic constant. So, my goal is to find \\( n \\) such that all these sums are equal.\n\nLooking at the Asymptote code provided, it seems like the magic square is a 3x3 grid. Let me visualize it based on the labels given:\n\n- The top row has three cells: \\( n-3 \\), \\( 3 \\), and \\( n+1 \\).\n- The middle row has \\( n+2 \\), \\( 2n-9 \\), and \\( 1 \\).\n- The bottom row has \\( 2 \\), \\( n \\), and \\( n-1 \\).\n\nSo, writing this out, the magic square looks like this:\n\n\\[\n\\begin{array}{|c|c|c|}\n\\hline\nn - 3 & 3 & n + 1 \\\\\n\\hline\nn + 2 & 2n - 9 & 1 \\\\\n\\hline\n2 & n & n - 1 \\\\\n\\hline\n\\end{array}\n\\]\n\nAlright, now I need to find \\( n \\) such that all rows

In [9]:
len(original_tokens)

1959

In [14]:
labels[:, -PAD_SIZE - 1 :][:, :max_correct]

tensor([[32313,    11,   773,   358,   614,   419, 10963]], device='cuda:0')